# Part 3: Hands-On Project

1. Write the core functionality of your addon in the main script file
   (e.g., `r.simwe.senario.py`).
2. Use GRASS libraries and tools to interact with GRASS data and perform
   operations.
3. Follow best practices for coding style and documentation.
4. Implement error handling and input validation.
5. Write unit tests to ensure the addon works as expected.
6. Write documentation for the addon, including usage instructions and examples.

In [3]:
%%bash
# cookiecutter https://github.com/cwhite911/grass-addon-cookiecutter --checkout grass-tools
# When using uv as a package manager, uncomment the following line:
# uv run cookiecutter https://github.com/cwhite911/grass-addon-cookiecutter --checkout grass-tools

- You've downloaded /home/coreywhite/.cookiecutters/grass-addon-cookiecutter before. Is it okay to delete and re-download it? [y/n] (y): y
  - [1/4] tool (Addon name): r.simwe.scenario           
  - [2/4] author_name (Your name (or your organization/company/team)): Corey T. White
  - [3/4] email (Your email): smortopahri@gmail.com
  - [4/4] description (A short description of the project.): Run overland flow scenarios with the SIMWE model.

Let migrate our existing script into this new addon structure.

In [ ]:
#!/usr/bin/env python

##############################################################################
# MODULE:    r.simwe.scenario
#
# AUTHOR(S): Corey T. White <smortopahri@gmail.com>
#
# PURPOSE:   Run overland flow scenarios with the SIMWE model.
#
# COPYRIGHT: (C) 2025 by Corey T. White and the GRASS Development Team
#
#            This program is free software under the GNU General Public
#            License (>=v2). Read the file COPYING that comes with GRASS
#            for details.
##############################################################################

"""Run overland flow scenarios with the SIMWE model."""

# %module
# % description: Run overland flow scenarios with the SIMWE model.
# % keyword: raster
# % keyword: hydrology
# % keyword: water
# %end
# %option G_OPT_R_ELEV
# %required: yes
# %end
# %option G_OPT_R_OUTPUT
# %end

import sys
import atexit
import grass.script as gs
from grass.tools import Tools
from gettext import gettext as _


def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)


def main():
    # initalize tools
    tools = Tools()

    # get input options
    options, flags = gs.parser()
    # elevation = options["elevation"]
    output_raster = options["output"]

    # crete a temporary raster that will be removed upon exit
    temporary_raster = gs.append_node_pid("gauss")
    atexit.register(clean, temporary_raster)

    # if changing computational region is needed, uncomment
    # Use RegionManager/MaskManager as context managers for temporary region/mask:
    # They automatically restore the previous region/mask on exit.
    # Example:
    # with gs.RegionManager():          # temporarily set computational region
    #     with gs.MaskManager():        # temporarily set raster/vector mask
    #         # perform region/mask-specific operations here
    #         pass

    # verbose message with translatable string
    gs.verbose(_("Generating temporary raster {tmp}").format(tmp=temporary_raster))

    # run analysis
    tools.r_surf_gauss(output=temporary_raster)

    # save history into the output raster
    gs.raster_history(output_raster, overwrite=True)


if __name__ == "__main__":
    sys.exit(main())